In [ ]:
import pandas as pd
import numpy as np
import warnings
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
fecha = f"{año}{str(mes).zfill(2)}" # 202401

nombre_base = f"base_asignacion_202411.xlsx" # Modificar
asignacion_path = f"bases/asignacion/{fecha}/{nombre_base}"
asignacion_pagos_path = f"bases/asignacion/{fecha}/base_asignacion_{mes_año}.xlsx"
asignacion_camp_path = f"bases/asignacion/{fecha}/base_asignacion_{mes_año}_CAMP.xlsx"

print(asignacion_path)
print(asignacion_pagos_path)
print(asignacion_camp_path)

df_asignacion = pd.read_excel(asignacion_path, sheet_name='Hoja1')
print("Base asignación pagos:", df_asignacion.shape)

In [ ]:
df_asignacion_test = df_asignacion.copy()

def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA_CORRECTA' in df.columns:
        df['AGENCIA_CORRECTA'].replace(' ', '', inplace=True)
        
        if 'AGENCIA' not in df.columns:
            df.rename(columns={'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)
        else:
            df['AGENCIA'] = np.where(
                df['AGENCIA'].notna() & (df['AGENCIA'].astype(str).str.len() > 0),
                df['AGENCIA'],
                df['AGENCIA_CORRECTA']
            )
            df.drop(columns=['AGENCIA_CORRECTA'], inplace=True)
    
    if 'CONTRATO' in df.columns:
        df.drop(columns=['CONTRATO'], inplace=True)
    
    if 'ID_VTA' not in df.columns:
        df['ID_VTA'] = np.nan
    
    if 'CAMPAÑA' not in df.columns:
        df['CAMPAÑA'] = np.nan
    
    df.rename(columns={'CODIGO': 'CC', 'CONT_18': 'CONTRATO', 'NOMBRE': 'NOMBRE_CLIENTE', 'CLAVE': 'CARTERA'}, inplace=True)
    
    return df

def update_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = rename_columns(df)
    
    df['TIPO_FONDO'].fillna('NULL', inplace=True)
    df['ID_VTA'].fillna('NULL', inplace=True)
    df['ID_VTA'].replace('-', 'NULL', inplace=True)
    
    df['CONTRATO'] = df['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    df['CC'] = df['CC'].astype('Int64').astype(str).str.zfill(8)
    
    conteo_cc = df['CC'].value_counts()
    df['FLAG'] = df['CC'].map(conteo_cc)
    
    df.sort_values(by=['CC', 'CONTRATO', 'CAPITAL'], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df[cols]

In [ ]:
cols_required = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'FLAG', 'OFICINA', 'TERRITORIO', 'ID_VTA']

df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)
df_asignacion_test = update_columns(df_asignacion_test, cols_required)
df_asignacion_test.head(5)

In [ ]:
cols_asig_pagos = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'FLAG']
df_asignacion_pagos = df_asignacion_test[cols_asig_pagos]

cols_asig_camp = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'OFICINA', 'TERRITORIO', 'ID_VTA']
df_asignacion_camp = df_asignacion_test[cols_asig_camp]

In [ ]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base asignacion para campañas?", icon='warning')

df_asignacion_pagos.to_excel(asignacion_pagos_path, index=False, sheet_name='BASE')
print("Base asignación pagos:", df_asignacion_pagos.shape)

if result == 'yes':
    df_asignacion_camp.to_excel(asignacion_camp_path, index=False, sheet_name='BASE')
    print("Base asignación campañas:", df_asignacion_camp.shape)

root.destroy()